# 🚀 Otomatik JetX Model Eğitmeni

Bu not defteri, mevcut tüm JetX tahmin modellerini, en iyi performans için optimize edilmiş hiperparametrelerle otomatik olarak eğitir.

## ✨ Özellikler
- **Tam Otomatik**: Manuel müdahale gerekmez.
- **Optimize Edilmiş Parametreler**: Her model, yüksek doğruluk için önceden ayarlanmış parametrelerle eğitilir.
- **Rolling Window Eğitimi**: Gerçek dünya senaryolarını simüle etmek için rolling window yaklaşımı kullanılır.
- **Detaylı Raporlama**: Her modelin eğitim süreci ve sonuçları net bir şekilde raporlanır.

**⚠️ Başlamadan Önce: Runtime > Change runtime type > GPU seçeneğini etkinleştirin!**

In [ ]:
# 🛠️ 1. Sistem Kurulumu
import sys
import os
import subprocess
import importlib.util

def check_and_install(package, import_name=None):
    """Paketi kontrol et ve yoksa yükle"""
    if import_name is None:
        import_name = package
    if importlib.util.find_spec(import_name) is None:
        print(f'📦 {package} yükleniyor...')
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
    else:
        print(f'✅ {package} zaten yüklü.')

required_packages = [("ipywidgets", "ipywidgets"), ("tqdm", "tqdm"), ("plotly", "plotly"), ("seaborn", "seaborn")]
for package, import_name in required_packages:
    check_and_install(package, import_name)

print("🎉 Kurulum tamamlandı!")

In [ ]:
# 📚 2. Gerekli Kütüphanelerin ve Projenin Yüklenmesi
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def clone_or_update_repo():
    """Repository'yi klonla veya güncelle"""
    repo_url = "https://github.com/onndd/predictor_1.git"
    project_dir = "/content/predictor_1"
    if os.path.exists(project_dir):
        print("📁 Proje zaten mevcut. Güncelleniyor...")
        os.chdir(project_dir)
        subprocess.run(["git", "pull", "origin", "main"], check=True, capture_output=True)
    else:
        print("📥 GitHub'dan proje klonlanıyor...")
        os.chdir("/content")
        subprocess.run(["git", "clone", repo_url], check=True, capture_output=True)
        os.chdir(project_dir)
    
    src_path = os.path.join(project_dir, "src")
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
        print(f"✅ {src_path} Python path'e eklendi")
    return project_dir

try:
    PROJECT_DIR = clone_or_update_repo()
    models_dir = "/content/trained_models"
    os.makedirs(models_dir, exist_ok=True)
    print(f"💾 Model dizini hazır: {models_dir}")
    print("🎉 Repository kurulumu tamamlandı!")
except Exception as e:
    print(f"❌ Repository kurulum hatası: {e}")

In [ ]:
# 📊 3. Veri Yükleme ve Hazırlama
from data_processing.loader import load_data_from_sqlite

def load_real_jetx_data():
    repo_db_path = "/content/predictor_1/data/jetx_data.db"
    if os.path.exists(repo_db_path):
        data = load_data_from_sqlite(repo_db_path)
        if data and len(data) > 1000:
            print(f"✅ Repository'den {len(data)} kayıt yüklendi")
            return data
    print("⚠️ Gerçek veri bulunamadı!")
    return None

def create_rolling_chunks(data, chunk_size=1000):
    chunks = []
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        if len(chunk) >= chunk_size:
            chunks.append(chunk)
    return chunks

print("📊 Gerçek JetX verisi yükleniyor...")
jetx_data = load_real_jetx_data()
if jetx_data:
    ROLLING_CHUNKS = create_rolling_chunks(jetx_data, 1000)
    print(f"📊 Rolling window chunks: {len(ROLLING_CHUNKS)} adet (her biri 1000 kayıt)")
    print("🎯 Veri hazırlama tamamlandı!")
else:
    print("❌ Veri hazırlama başarısız!")

In [ ]:
# 🏗️ 4. Eğitim Sisteminin Yüklenmesi
exec(open('/content/predictor_1/colab/rolling_training_system.py').read())
exec(open('/content/predictor_1/colab/rolling_integration.py').read())

model_registry = ModelRegistry()
print("✅ Rolling Window Eğitim Sistemi ve Model Kayıtçısı hazır!")

In [ ]:
# 🚀 5. Otomatik Eğitim ve Raporlama
import traceback

OPTIMIZED_PARAMS = {
    'N-Beats': {
        'sequence_length': 200,
        'hidden_size': 512,
        'num_stacks': 4,
        'num_blocks': 4,
        'learning_rate': 0.0005,
        'epochs_per_cycle': 50,
        'batch_size': 32,
        'threshold': 1.5,
        'crash_weight': 2.5
    },
    'TFT': {
        'sequence_length': 200,
        'hidden_size': 256,
        'num_heads': 4,
        'num_layers': 3,
        'learning_rate': 0.0005,
        'epochs_per_cycle': 40,
        'batch_size': 16,
        'threshold': 1.5,
        'crash_weight': 2.5
    },
    'LSTM': {
        'sequence_length': 150,
        'lstm_units': 256,
        'dropout_rate': 0.3,
        'learning_rate': 0.0005,
        'epochs_per_cycle': 60,
        'batch_size': 32,
        'threshold': 1.5,
        'crash_weight': 2.5
    }
}

def run_automated_training(model_registry, rolling_chunks):
    print("🚀 BAŞLATILIYOR: Otomatik Model Eğitimi")
    print("="*60)
    
    models_to_train = ['N-Beats', 'TFT', 'LSTM']
    
    for model_name in models_to_train:
        print(f"\n\n--- {model_name} EĞİTİMİ BAŞLIYOR ---")
        params = OPTIMIZED_PARAMS[model_name]
        print("⚙️ Kullanılan Parametreler:")
        for key, value in params.items():
            print(f"   - {key}: {value}")
        
        try:
            trainer = RollingWindowTrainer(model_registry, rolling_chunks)
            trainer.start_rolling_training(
                model_type=model_name,
                sequence_length=params['sequence_length'],
                hidden_size=params.get('hidden_size', params.get('lstm_units')),
                num_stacks=params.get('num_stacks'),
                num_blocks=params.get('num_blocks'),
                num_heads=params.get('num_heads'),
                num_layers=params.get('num_layers'),
                dropout_rate=params.get('dropout_rate'),
                learning_rate=params['learning_rate'],
                epochs_per_cycle=params['epochs_per_cycle'],
                batch_size=params['batch_size'],
                threshold=params['threshold'],
                crash_weight=params['crash_weight']
            )
            print(f"✅ {model_name} eğitimi başarıyla tamamlandı.")
        except Exception as e:
            print(f"❌ {model_name} eğitimi sırasında bir hata oluştu: {e}")
            traceback.print_exc()
        print(f"--- {model_name} EĞİTİMİ BİTTİ ---")

    print("\n\n🎉 TÜM MODELLERİN EĞİTİMİ TAMAMLANDI 🎉")
    print("="*60)
    
    print("\n📊 EĞİTİM SONUÇLARI ÖZETİ:")
    analyze_training_results()

if 'ROLLING_CHUNKS' in globals() and 'model_registry' in globals() and ROLLING_CHUNKS:
    run_automated_training(model_registry, ROLLING_CHUNKS)
else:
    print("❌ Otomatik eğitim başlatılamadı. Veri yükleme adımının başarılı olduğundan emin olun.")